In [5]:
# coding: utf-8
# In[110]:

from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances


import pandas as pd
get_ipython().magic('matplotlib')


# In[111]:

df_all=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')

df_grouped=df_all.groupby(['patient_nbr'], sort=True).last()

print("Shape grouped: ",df_grouped.shape)
#print(df_grouped)

#df_all=pd.read_csv('C:/gcat_epi_genomic.csv')
#df=pd.read_csv('C:/binary_2016_11_14.csv')

#df_all = df_all[df_all['ENFERMEDADES_DIABETES_T2DM']==1]
msk = np.random.rand(len(df_grouped)) < 0.8
df, df_test = df_grouped[msk].copy(deep = True), df_grouped[~msk].copy(deep = True)
df = df.reset_index()
df_test = df_test.reset_index()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)

print("Columns: ",df.columns)
y_train=df['readmitted']
y_test=df_test['readmitted']

print("Test", set(y_test))

# In[112]:

to_del = ['admission_source_id','encounter_id', 'patient_nbr','medical_specialty','payer_code','index','admission_type_id','discharge_disposition_id',
          'nateglinide','chlorpropamide','acetohexamide','tolbutamide','acarbose','miglitol','troglitazone','tolazamide','examide','citoglipton','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone']        

print (to_del)

#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
df_2_test = df_test[filtered_cols]
print ("df_2",df_2.shape)

from sklearn.model_selection import train_test_split
import numpy as np
df_3 = df_2.fillna(value=np.mean(df_2,axis=0),inplace=False,axis=0).values
df_3_test = df_2_test.fillna(value=np.mean(df_2,axis=0),inplace=False,axis=0).values
print ("df_3",df_3.shape)
print(df_2.columns)
x_train=df_3[:,0:-1]
x_test=df_3_test[:,0:-1]


y_train=np.where(y_train > 0, 1, y_train)
y_test=np.where(y_test > 0, 1, y_test)
print(set(y_train))
print(set(y_test))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

from imblearn.over_sampling import SMOTE
sm = SMOTE(n_jobs=-1, random_state=42,kind='regular')
x_train_res,y_train_res = sm.fit_sample(x_train,y_train)
x_test_res,y_test_res = sm.fit_sample(x_test,y_test)

print(x_train_res.shape)
print(x_test_res.shape)
print(y_train_res.shape)
print(y_test_res.shape)

Using matplotlib backend: Qt5Agg
Shape grouped:  (71518, 49)
df.shape (57444, 50)
df_test.shape (14074, 50)
Columns:  Index(['patient_nbr', 'encounter_id', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'di

In [2]:
from sklearn.preprocessing import StandardScaler
# Define the scaler 
scaler = StandardScaler().fit(x_train_res)
# Scale the train set
x_train = scaler.transform(x_train_res)
# Scale the test set
x_test = scaler.transform(x_test_res)
y_train=y_train_res
y_test=y_test_res

In [3]:
#Random Forest
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=2, oob_score = True) 

# Set the parameters by cross-validation
#'min_samples_leaf': [1,5,10,50,100,200,500],

tuned_parameters = {
    'n_estimators': [30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight':['balanced']
}

#tuned_parameters = {  
#    }  

#scores = ['precision', 'recall', 'f1']
#scores = [ 'f1_weighted','f1_micro','f1_macro','accuracy','roc_auc']
scores=['f1_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    #clf = GridSearchCV(GaussianNB(), tuned_parameters, cv=10,scoring='%s_macro' % score)
    #clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s_weighted' % score,n_jobs= -1)
    clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s' % score,n_jobs= -1)
    clf.fit(x_train, y_train)
    array=clf.best_estimator_.feature_importances_
    print(type(array))
    for i in range (0,len(array)):
        print(df_2.columns[i],": ",array[i])
    #print("clf: ",type(clf))
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_train, y_pred = y_test, clf.predict(x_test)
    y_train2, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_train2, y_pred))
    print("Accuracy score: ", accuracy_score(y_test, y_pred))
    print()

# Tuning hyper-parameters for f1_weighted

<class 'numpy.ndarray'>
race :  0.0311062832966
gender :  0.0533411088014
age :  0.0393533447394
weight :  0.0176718309997
time_in_hospital :  0.047652660529
num_lab_procedures :  0.0623444646118
num_procedures :  0.0310626626258
num_medications :  0.0537201711642
number_outpatient :  0.0489685983937
number_emergency :  0.0233211784791
number_inpatient :  0.112781533486
diag_1 :  0.0569782669627
diag_2 :  0.0559994238994
diag_3 :  0.0549728476395
number_diagnoses :  0.0448199756492
max_glu_serum :  0.00827659738645
A1Cresult :  0.015249401108
metformin :  0.029103979134
repaglinide :  0.00209880823781
glimepiride :  0.00638336973105
glipizide :  0.0245143926596
glyburide :  0.013102836332
pioglitazone :  0.0102089876425
rosiglitazone :  0.00765140648611
insulin :  0.0423565180526
change :  0.0638383575944
diabetesMed :  0.0431209943582
Best parameters set found on development set:

{'class_weight': 'balanced', 'max_features': 'log2', 'n_estima